***stock Evaluation***
********************************************************************************
**This script aims to evaluate, based on the hystorical date of an stock, how much is convenient to reduce the occurence of the investment in the stock**

***Libraries Definition***

In [171]:
##Generic library for Array and Data-time format
import datetime as dt
import time
import glob
import math
import os
import numpy as np
import pandas as pd

pd.set_option("future.no_silent_downcasting", True)

##Generic library to create plots
import plotly.graph_objects as go
import plotly.subplots as sp

##Generic library to retrieve stock-Data
import yfinance as yf

GENERAL PURPOSE FUNCTIONS
=========================

In [12]:
##Return the DATA INFORMATIONS
def month_year():
    now = dt.datetime.now()
    return now.day, now.month, now.year

Here we have the function that retrieve the stock information based on the choosen ISIN

Generic Functions to retrieve Stock Data from Database
======================================================

In [210]:
##Return the yfinance.Ticker object that stores all the relevant stock informations
def get_stock_data(isin_string):
    stock_ticker = isin_string
    stock_data = yf.Ticker(stock_ticker)
    stock_data.info
    return stock_data


##Return the hystorical data with date expressed as string --> Suitable for calculations
def get_stock_with_date_index_data(stock_data, start_date, end_date):
    try:
        hist_data = stock_data.history(start=start_date, end=end_date)
        hist_data = hist_data["Close"]
        hist_data.index = hist_data.index.strftime("%Y-%m-%d")
        # Genera un nuovo indice che copre l'intero intervallo data
        full_date_range = pd.date_range(start=start_date, end=end_date, freq="D")
        # Reindirizza i dati per includere il nuovo indice
        hist_data = hist_data.reindex(full_date_range.strftime("%Y-%m-%d"))
        # Applica forward fill per riempire i valori mancanti
        hist_data.ffill(inplace=True)
        hist_data_to_return = pd.DataFrame(
            index=full_date_range.strftime("%Y-%m-%d"), columns=["stock_price"]
        )
        hist_data_to_return["stock_price"] = hist_data
        print(f"get_stock_with_date : {hist_data_to_return}")
        return hist_data_to_return
    except Exception as e:
        print(f"Error: {e}")
        return None

Functions Related to Investment Strategies
=========================================

In [231]:
def get_info_investment(
    stock_data, initial_capital, start_date, end_date, purchase_frequency
):
    # Crea un intervallo di date dal start_date al end_date con purchase_frequency interval
    purchase_dates = pd.date_range(start_date, end_date, freq=purchase_frequency)
    purchase_dates = purchase_dates.strftime("%Y-%m-%d")
    # Crea un DataFrame vuoto che coprirà ogni giorno tra start_date e end_date
    daily_investment_df = pd.DataFrame(
        index=(pd.date_range(start=start_date, end=end_date, freq="D")).strftime(
            "%Y-%m-%d"
        ),
        columns=[
            "price",
            "shares_bought",
            "average_cost",
            "total_investment",
            "total_shares",
            "daily_stock_price",
        ],
    )
    ## print(f"initialized daily_investment_df {daily_investment_df}")

    total_investment = 0
    total_shares = 0

    # Ciclo attraverso ogni data di acquisto
    for date in purchase_dates:
        purchase_amount = initial_capital / len(purchase_dates)
        total_investment += purchase_amount
        daily_stock_price = stock_data.loc[date, "stock_price"]
        shares_bought = purchase_amount / daily_stock_price if daily_stock_price else 0
        total_shares += shares_bought
        # Imposta i valori per il giorno di acquisto
        daily_investment_df.loc[date] = [
            daily_stock_price,
            shares_bought,
            total_investment / total_shares if total_shares else 0,
            total_investment,
            total_shares,
            daily_stock_price,
        ]

    # Riempie in avanti i giorni senza acquisti con i valori dell'ultimo acquisto noto
    daily_investment_df.ffill(inplace=True)

    # Stock price between purchase_dates
    daily_investment_df["daily_stock_price"] = stock_data["stock_price"]

    # Calcola i valori di mercato giornalieri e i guadagni
    daily_investment_df["market_value"] = (
        daily_investment_df["daily_stock_price"] * daily_investment_df["total_shares"]
    )
    daily_investment_df["daily_gain"] = (
        daily_investment_df["market_value"] - daily_investment_df["total_investment"]
    )
    daily_investment_df["daily_gain_perc"] = (
        daily_investment_df["daily_gain"] / daily_investment_df["total_investment"]
    ) * 100

    # Riempe in NAN
    daily_investment_df.ffill(inplace=True)

    final_data = {
        "average_cost": daily_investment_df["average_cost"],
        "market_value": daily_investment_df["market_value"],
        "daily_gains_df": daily_investment_df,
        "total_shares": daily_investment_df["total_shares"],
        "total_investment": daily_investment_df["total_investment"],
        "dates_purchase_list": daily_investment_df.index.tolist(),
    }

    return final_data


# Find which is the best strategy of investment
def get_best_investment_strategy(results):
    best_strategy = None
    best_average_cost = float("inf")
    best_number_shares = 0
    best_return_value = 0
    best_market_value = 0

    for freq, result in results.items():
        average_cost = result["average_cost"].iloc[-1]
        number_shares = result["total_shares"].iloc[-1]
        last_date_purchase = result["dates_purchase_list"][-1]  ##It is a list
        final_return_value = result["daily_gains_df"]["daily_gain"].iloc[-1]
        market_value = result["market_value"].iloc[-1]

        if average_cost < best_average_cost:
            best_strategy = freq
            best_average_cost = average_cost
            best_number_shares = number_shares
            best_last_date_purchase = last_date_purchase
            best_return_value = final_return_value
            best_market_value = market_value
    print(
        f"The winning strategy is {best_strategy} with an average cost of {best_average_cost:.2f} , {best_number_shares:.2f} shares and last purchase on {best_last_date_purchase} with a return value of {best_return_value} USD."
    )
    print(f"The final market value at is : {best_market_value} USD")
    return (best_strategy, best_average_cost, best_number_shares, best_market_value)


GENERIC PLOT FUNCTIONS
===========================

In [174]:
##To be used in order to plot the stock behavior along two dates that you choose
def plot_stock_data(stock_data, start_date, end_date):
    try:
        hist_data = stock_data.history(start=start_date, end=end_date)
        fig = go.Figure(data=[go.Scatter(x=hist_data.index, y=hist_data["Close"])])
        fig.update_layout(
            title=f"stock <b>{stock_data.ticker}</b> Price History",
            xaxis_title="Date",
            yaxis_title="Price (USD)",
        )
        fig.show()
    except Exception as e:
        print(f"Error: {e}")


def create_plot(x, y, name_trace, name_graph, xaxis_title, yaxis_title):
    fig = go.Figure()
    for x_list, y_list, name in zip(x, y, name_trace):
        fig.add_trace(
            go.Scatter(
                x=x_list,
                y=y_list,
                mode="lines+markers+text",
                name=name,
            )
        )
    fig.update_layout(
        title=name_graph,
        xaxis_title=xaxis_title,
        yaxis_title=yaxis_title,
        legend_title="Legenda",
        hovermode="x",
    )
    fig.show()
    return fig

***MAIN CODE***
===========================

Hystorical Behavior of the Selected stock
===========================

In [132]:
### GENERIC INFORMATION ABOUT stock and its HYSTORICAL BEHAVIOR
# isin_string = "IE00B4L5Y983", ticker_string = "VWRA.L", ticker_string = "GME"
stock_under_test = input("Enter the stock ticker symbol: ")
# Time Informations
day, month, year = month_year()
start_date = input("Enter the start date (YYYY-MM-DD): ")
# end_date = input("Enter the end date (YYYY-MM-DD): ")
end_date = f"{year}-{month}-{day}"
stock_data_object = get_stock_data(stock_under_test)
stock_data_daily_values = plot_stock_data(stock_data_object, start_date, end_date)

Evaluation with DCA with Custom Purchase Frequency
===========================

User Input Data
===============

In [223]:
initial_capital = float(input("Insert how much you want to invest in USD"))
start_date_dca = input("Enter the start date for DCA Strategy (YYYY-MM-DD): ")
end_date_dca = input("Enter the end date for DCA Strategy (YYYY-MM-DD): ")
# start_date_dca = "2020-05-18"
# end_date_dca = "2021-06-25"
stock_data_dca_values = get_stock_with_date_index_data(
    stock_data_object, start_date_dca, end_date_dca
)
# end_date_dca = f"{year}-{month}-{day}"  ##Today
# end_date_dca = "2024-12-31"
purchase_frequencies = [
    "1ME",
    "3ME",
    "6ME",
    "9ME",
    "12ME",
    "15ME",
]  # range of purchase frequencies
# purchase_frequencies = [
#    "12ME",
# ]

get_stock_with_date :             stock_price
2018-05-18         1.63
2018-05-19         1.63
2018-05-20         1.63
2018-05-21         1.73
2018-05-22         1.66
...                 ...
2024-06-07         1.07
2024-06-08         1.07
2024-06-09         1.07
2024-06-10         1.09
2024-06-11         1.09

[2217 rows x 1 columns]


Calculations
========================

In [232]:
results = {}

for freq in purchase_frequencies:
    (result) = get_info_investment(
        stock_data_dca_values,
        initial_capital,
        start_date_dca,
        end_date_dca,
        freq,
    )
    results[freq] = result
    #

daily_stock_price 2018-05-18    1.63
2018-05-19    1.63
2018-05-20    1.63
2018-05-21    1.73
2018-05-22    1.66
              ... 
2024-06-07    1.07
2024-06-08    1.07
2024-06-09    1.07
2024-06-10    1.09
2024-06-11    1.09
Name: daily_stock_price, Length: 2217, dtype: float64
daily_stock_price 2018-05-18    1.63
2018-05-19    1.63
2018-05-20    1.63
2018-05-21    1.73
2018-05-22    1.66
              ... 
2024-06-07    1.07
2024-06-08    1.07
2024-06-09    1.07
2024-06-10    1.09
2024-06-11    1.09
Name: daily_stock_price, Length: 2217, dtype: float64
daily_stock_price 2018-05-18    1.63
2018-05-19    1.63
2018-05-20    1.63
2018-05-21    1.73
2018-05-22    1.66
              ... 
2024-06-07    1.07
2024-06-08    1.07
2024-06-09    1.07
2024-06-10    1.09
2024-06-11    1.09
Name: daily_stock_price, Length: 2217, dtype: float64
daily_stock_price 2018-05-18    1.63
2018-05-19    1.63
2018-05-20    1.63
2018-05-21    1.73
2018-05-22    1.66
              ... 
2024-06-07    1.07
2024-0

Plotting Section for DCA vs Sum Lump 
====================================

In [225]:
plots = []

# Assumiamo che stock_data_dca_values sia un DataFrame con una colonna 'price'
# e che l'indice sia un DateTimeIndex delle date

# Aggiungiamo il primo plot, che rappresenta il prezzo e il costo medio di carico

plots.append(
    {
        "y": [stock_data_dca_values["stock_price"]]
        + [result["average_cost"] for result in results.values()],
        "x": [stock_data_dca_values.index]
        + [result["dates_purchase_list"] for result in results.values()],
        "name_trace": ["Stock Price"]
        + [f"Average Cost with {freq}" for freq in purchase_frequencies],
        "name_graph": "Stock Price and Average Cost with DCA",
        "xaxis_title": "Date",
        "yaxis_title": "Price in USD",
    }
)

# Aggiungiamo gli altri plot usando un ciclo for per estrarre i dati da ciascun risultato
for metric in [
    ("market_value", "Daily Market Value with DCA", "Price in USD"),
    ("daily_gain", "Daily Gain with DCA", "Price in USD"),
    ("daily_gain_perc", "Daily Gain Percentage with DCA", "Gain % vs Investment"),
]:
    plot_data = {
        "y": [
            results[freq]["daily_gains_df"][metric[0]] for freq in purchase_frequencies
        ],
        "x": [results[freq]["dates_purchase_list"] for freq in purchase_frequencies],
        "name_trace": [f"{metric[1]} {freq}" for freq in purchase_frequencies],
        "name_graph": f"{metric[1]} Graph",
        "xaxis_title": "Date",
        "yaxis_title": metric[2],
    }
    plots.append(plot_data)

# Qui andrebbe definita la funzione 'create_plot'
# Per ogni configurazione di plot nel nostro array, generiamo e visualizziamo il grafico
for plot in plots:
    fig = create_plot(
        x=plot["x"],
        y=plot["y"],
        name_graph=plot["name_graph"],
        name_trace=plot["name_trace"],
        xaxis_title=plot["xaxis_title"],
        yaxis_title=plot["yaxis_title"],
    )


##Define which is the best strategy
(best_strategy, best_average_cost, best_number_shares, best_market_value) = (
    get_best_investment_strategy(results)
)

#### Strategia con 200MA

### Controllo la 200MA per capire se investire o meno
### Se il mio prezzo medio e' sopra la 200MA e la 200MA cresce allora non faccio niente, altrimenti quando average_cost <= 200MA, compro


The winning strategy is 1ME with an average cost of 1.34 , 7460.10 shares and last purchase on 2024-06-11 with a return value of -1868.4925857567177 USD.
The final market value at is : 8131.507414243293 USD
